In [1]:
import os
import sys
os.chdir('../')
sys.path.append('../')

import torch
from utils.utils import train,prepare,evaluate
from models.Encoders import NRMS_Encoder, FIM_Encoder, NPA_Encoder, Pipeline_Encoder, MHA_Encoder, Bert_Encoder, RNN_Encoder, CNN_Encoder
from models.Interactors import FIM_Interactor, KNRM_Interactor
from models.SFI import SFI_gating

In [2]:
name='sfi'
hparams = {
    'scale':'demo',
    'mode':'train',
    'device':'cuda:0',
    'epochs':8,
    'batch_size':50,
    'his_size':20,
    'k':9,
    'contra_num':3,
    'title_size':20,
    'abs_size':40,
    'npratio':4,
    'dropout_p':0.2,
    'embedding_dim':300,
    'value_dim':16,
    'query_dim':200,
    'head_num':16,
    'filter_num':150,
    'select':'gating',
    'level':1,
    'metrics':'auc,mean_mrr,ndcg@5,ndcg@10',
    # 'attrs': ['abs','vert','subvert'],
    'attrs': ['title'],
    'save_step':[0],
    'spadam':True,
    # 'dynamic':True,
}
# torch.cuda.set_device(hparams['device'])

In [3]:
hparams['validate'] = True
vocab, loaders = prepare(hparams)

[2021-04-15 03:10:53,025] INFO (root) Hyper Parameters are
{'scale': 'demo', 'mode': 'train', 'device': 'cuda:0', 'epochs': 8, 'batch_size': 50, 'his_size': 20, 'k': 9, 'contra_num': 3, 'title_size': 20, 'abs_size': 40, 'npratio': 4, 'dropout_p': 0.2, 'embedding_dim': 300, 'value_dim': 16, 'query_dim': 200, 'head_num': 16, 'filter_num': 150, 'select': 'gating', 'level': 1, 'metrics': 'auc,mean_mrr,ndcg@5,ndcg@10', 'attrs': ['title'], 'save_step': [0], 'spadam': True, 'validate': True}
[2021-04-15 03:10:53,027] INFO (root) preparing dataset...
[2021-04-15 03:10:57,062] INFO (torchtext.vocab) Loading vectors from .vector_cache/glove.840B.300d.txt.pt


In [4]:
record = next(iter(loaders[0]))

In [4]:
encoder = FIM_Encoder(hparams, vocab)
interactor = FIM_Interactor(hparams['k'], encoder.signal_length)

hparams['name'] = '-'.join([name,encoder.name,hparams['select']])

sfi = SFI_gating(hparams, encoder, interactor).to(hparams['device'])

# sfi.load_state_dict(torch.load('/home/peitian_zhang/Codes/News-Recommendation/data/model_params/sfi-fim-encoder-gating/demo_epoch8_step0_[hs=20,topk=9,attrs=title].model')['model'])

In [6]:
sfi(record)

tensor([[-1.5063, -1.5821, -1.6571, -1.6414, -1.6695],
        [-1.5618, -1.6188, -1.5770, -1.7371, -1.5632],
        [-1.6679, -1.4857, -1.6347, -1.6347, -1.6347]], device='cuda:0',
       grad_fn=<LogSoftmaxBackward>)

In [5]:
sfi = train(sfi, hparams, loaders, interval=10)

[2021-04-15 03:11:22,657] INFO (root) training...
epoch 1 , step 50 , loss: 1.8344: : 59it [00:04, 12.92it/s]
[2021-04-15 03:11:29,425] INFO (root) saved model of step 0, epoch 1 at data/model_params/sfi-fim-encoder-gating/demo_epoch1_step0_[hs=20,topk=9,contra=3,attrs=title].model
epoch 2 , step 50 , loss: 1.6568: : 59it [00:03, 15.47it/s]
[2021-04-15 03:11:35,481] INFO (root) saved model of step 0, epoch 2 at data/model_params/sfi-fim-encoder-gating/demo_epoch2_step0_[hs=20,topk=9,contra=3,attrs=title].model
epoch 3 , step 50 , loss: 1.5478: : 59it [00:03, 15.08it/s]
[2021-04-15 03:11:41,634] INFO (root) saved model of step 0, epoch 3 at data/model_params/sfi-fim-encoder-gating/demo_epoch3_step0_[hs=20,topk=9,contra=3,attrs=title].model
epoch 4 , step 50 , loss: 1.4723: : 59it [00:03, 15.13it/s]
[2021-04-15 03:11:47,780] INFO (root) saved model of step 0, epoch 4 at data/model_params/sfi-fim-encoder-gating/demo_epoch4_step0_[hs=20,topk=9,contra=3,attrs=title].model
epoch 5 , step 50 

In [8]:
evaluate(sfi, hparams, loaders[1], loading=True)

[2021-04-12 15:16:03,337] INFO (root) Loading model from data/model_params/sfi-fim-encoder-gating/demo_epoch8_step0_[hs=20,topk=9,attrs=title].model...
[2021-04-12 15:16:03,339] INFO (root) 
evaluating...
221it [00:02, 95.26it/s]
Exception in thread Thread-16:
Traceback (most recent call last):
  File "/home/peitian_zhang/anaconda3/envs/nn/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/home/peitian_zhang/anaconda3/envs/nn/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/home/peitian_zhang/anaconda3/envs/nn/lib/python3.8/site-packages/torch/utils/data/_utils/pin_memory.py", line 25, in _pin_memory_loop
    r = in_queue.get(timeout=MP_STATUS_CHECK_INTERVAL)
  File "/home/peitian_zhang/anaconda3/envs/nn/lib/python3.8/multiprocessing/queues.py", line 116, in get
    return _ForkingPickler.loads(res)
  File "/home/peitian_zhang/anaconda3/envs/nn/lib/python3.8/site-packages/torch/multiprocessing/reduction

KeyboardInterrupt: 

In [ ]:
hparams['name'] = '-'.join([name,'nrms',hparams['select']])
nrms_encoder = NRMS_Encoder(hparams, vocab)
sfi = SFI_gating(hparams, nrms_encoder).to(hparams['device'])

In [ ]:
sfi = train(sfi, hparams, loaders, interval=10)

In [ ]:
hparams['name'] = '-'.join([name,'npa',hparams['select']])

hparams['user_dim'] = 200
hparams['query_dim'] = 200
hparams['filter_num'] = 400

npa_encoder = NPA_Encoder(hparams, vocab, len(loaders[0].dataset.uid2index))
sfi = SFI_gating(hparams, npa_encoder).to(hparams['device'])

In [ ]:
sfi = train(sfi, hparams, loaders, interval=10)

In [ ]:
hparams['name'] = '-'.join([name,'pipeline',hparams['select']])

hparams['pipeline'] = 'sfi-fim'
# hparams['scale'] = 'large'

ppl_encoder = Pipeline_Encoder(hparams)
sfi = SFI_gating(hparams, ppl_encoder).to(hparams['device'])

In [ ]:
sfi = train(sfi, hparams, loaders, interval=10)

In [ ]:
hparams['name'] = '-'.join([name,'mha',hparams['select']])

mha_encoder = MHA_Encoder(hparams, vocab)
sfi = SFI_gating(hparams, mha_encoder).to(hparams['device'])

In [ ]:
sfi = train(sfi, hparams, loaders, interval=10)

In [ ]:
hparams['name'] = '-'.join([name,'lstm',hparams['select']])
hparams['hidden_dim'] = 256

rnn_encoder = RNN_Encoder(hparams,vocab)
sfi = SFI_gating(hparams, rnn_encoder).to(hparams['device'])

In [ ]:
sfi = train(sfi, hparams, loaders, interval=10)

In [ ]:
hparams['name'] = '-'.join([name,'bert',hparams['select']])

hparams['level'] = 2
# hparams['batch_size'] = 10
# hparams['bert'] = 'bert-base-uncased'
hparams['bert'] = 'albert-base-v2'

_, loaders = prepare(hparams)

In [ ]:
bert_encoder = Bert_Encoder(hparams)
sfi = SFI_gating(hparams, bert_encoder).to(hparams['device'])

In [ ]:
sfi = train(sfi, hparams, loaders, interval=10)

## The rest is for development